## 肺癌のセグメンテーションと良悪性の判別モデル

１．CTデータをpytorchで扱えるように

２．肺の腫瘍のみをセグメンテーション（タスクの焦点を絞るために）

３．関心ボクセルをグループ化して結節候補をまとめる

４．結節候補をConvalutonで分類する

５．結節ごとを患者ごとへの評価にする（今回は悪性度の最大値）

こぶの内悪性が結節
annotationsは結節フラグの大きさ
candidatesはすべてのこぶ情報

In [2]:
from collections import namedtuple
CandidateInfoTuple = namedtuple(
    'CandidateInfoTuple',
    'isNodule_bool, diameter_mm, series_uid, center_xyz',
)

In [3]:
CandidateInfoTuple

__main__.CandidateInfoTuple

In [5]:
#CTデータの内mhdファイルをリストにする
import functools

@functools.lru_cache(1)
def getCandidateInfoList(requireOnDisk_bool=True):
    mhd_list = glob.glob('data-unversioned/part2/luna/subset*/*.mhd')
    presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}

In [15]:
import csv


diameter_dict = {}
with open('annotations.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
            annotationDiameter_mm = float(row[4])

            diameter_dict.setdefault(series_uid, []).append(
                (annotationCenter_xyz, annotationDiameter_mm)
            )

In [16]:
#diameter_dict

#uid一つに複数のこぶがある場合もあり、こぶの中心座標と大きさがvalueに入る。

In [30]:
a="1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405"

In [38]:
#辞書のaの要素を返す。なければ台に引数を返す。デフォはNone
b=diameter_dict.get(a,[])
print(b)
s=diameter_dict.get("lll",[])
print(s)
f=diameter_dict.get("lll")
print(f)


[((-24.0138242, 192.1024053, -391.0812764), 8.143261683), ((2.441546798, 172.4648812, -405.4937318), 18.54514997), ((90.93171321, 149.0272657, -426.5447146), 18.20857028), ((89.54076865, 196.4051593, -515.0733216), 16.38127631)]
[]
None


In [20]:
candidateInfo_list = []
with open('candidates.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
#presentOnDisk_setはCTデータのuid
#そこに無いのはスキップでcontinue
            if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                continue

            isNodule_bool = bool(int(row[4]))
            candidateCenter_xyz = tuple([float(x) for x in row[1:4]])

            candidateDiameter_mm = 0.0
            for annotation_tup in diameter_dict.get(series_uid, []):
                annotationCenter_xyz, annotationDiameter_mm = annotation_tup
                for i in range(3):
                    delta_mm = abs(candidateCenter_xyz[i] - annotationCenter_xyz[i])
                    if delta_mm > annotationDiameter_mm / 4:
                        break
                else:
                    candidateDiameter_mm = annotationDiameter_mm
                    break

            candidateInfo_list.append(CandidateInfoTuple(
                isNodule_bool,
                candidateDiameter_mm,
                series_uid,
                candidateCenter_xyz,
            ))

NameError: name 'presentOnDisk_set' is not defined